# PRISM Data Processing

The purpose of this notebook is to process the PRISM data for historical moments. The data set extends from 1/1/1981 until the present.

- 1/1/1981 through 12/31/2018

The 30-yr, data focus period is:

- 1/1/1981 through 12/31/2010

PRISM data are provided on a 4 km grid and the values are daily values with days being referenced to UTC.

## Imports and Parameters

In [1]:
# this tells Jupyter to embed matplotlib plots in the notebook
%matplotlib notebook

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
import shapely as sp
from matplotlib.collections import PatchCollection
from matplotlib.lines import Line2D
from shapely.geometry import Point
from shapely.geometry import Polygon
from IPython.display import display, HTML
import os
from copy import deepcopy
import pyodbc
import sqlalchemy

Custom python code module

In [3]:
import DBA_DClimComp as DBAD

Output directory

In [4]:
OUT_DIR = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\Da' \
          r'ta\JNotes\Processed\PRISM'

All of the PRISM data have been placed in a database

For precipitation we have a wet threshold. The precipitation depth must equal or exceed this threshold before being counted as a wet day

In [23]:
WD_THRESH = 0.2   # in mm

## Processing and Aggregation to Monthly and Annual

Our first processing step for precipitation is to aggregate to monthly and calendar year annual values. These should be output to RData structures for seasonal and harmonic analysis in R, spreadsheets for manual examination, and Python pickle files for later loading into Jupyter notebooks.

Make the connection to the DB using a SQL Alchemy engine object

In [5]:
engine = sqlalchemy.create_engine( DBAD.DSN_STRING )

Acquire a Pandas DataFrame for our PRISM grid definition

In [6]:
GridSQL = DBAD.createSQLPRISMGrid()
GridDF = pd.read_sql( GridSQL, engine, index_col=DBAD.FIELDN_ID )

In [7]:
display( HTML( GridDF.to_html() ) )

,Grid_Index,Longitude,Lattitude,utm_x,utm_y
Id,,,,,
1,662328,-101.166664,30.291666,291619.21875,3353093.00
2,662329,-101.125000,30.291666,295627.46875,3353017.25
3,662330,-101.083336,30.291666,299635.65625,3352943.00
4,662331,-101.041664,30.291666,303643.78125,3352870.25
5,662332,-101.000000,30.291666,307651.87500,3352798.75
6,662333,-100.958336,30.291666,311659.90625,3352729.00
7,662334,-100.916664,30.291666,315667.87500,3352660.50
8,662335,-100.875000,30.291666,319675.81250,3352593.75
9,662336,-100.833336,30.291666,323683.71875,3352528.25


In [8]:
GridCols = list( GridDF.columns )
GridCols

['Grid_Index', 'Longitude', 'Lattitude', 'utm_x', 'utm_y']

Get our start and end dates and then go through our grid cells, read in the precipitation values for the specified time interval, generate monthly and yearly sums. 

Save all of our DataFrames in dictionaries by Grid Code.

In [26]:
START_DT = dt.datetime( 1981, 1, 1, 0, 0, 0 )
END_DT = dt.datetime( 2010, 12, 31, 0, 0, 0 )

In [27]:
DryDict = dict()
WetDict = dict()

In [28]:
NumGPts = len( GridDF )

In [29]:
for iI in range(1, (NumGPts + 1)):
    GridInd = int( GridDF.at[iI, GridCols[0]] )
    GridUTMX = float( GridDF.at[iI, GridCols[3]] )
    GridUTMY = float( GridDF.at[iI, GridCols[4]] )
    # now are ready to get our precipitation values
    PreSQL = DBAD.createSQLPRISMPre( START_DT, END_DT, iI )
    PreDF = pd.read_sql( PreSQL, engine, index_col=DBAD.FIELDN_STRDT, 
                         parse_dates=[DBAD.FIELDN_STRDT] )
    PreDF.index.name = DBAD.FIELDN_DT
    PreDF.index = PreDF.index.tz_convert( None )
    # now do the resample 
    MonDF = PreDF.resample( 'MS', axis=0, closed='left', label='left' ).sum()
    AnnDF = PreDF.resample( 'AS', axis=0, closed='left', label='left' ).sum()
    # change the column names
    MonDF.columns = ["Precip_mm"]
    AnnDF.columns = ["Precip_mm"]
    # now make our appends
    GMonDF = MonDF.copy()
    GMonDF.columns = ["%d" % GridInd]
    GAnnDF = AnnDF.copy()
    GAnnDF.columns = ["%d" % GridInd]
    # now check where we are
    if iI == 1:
        AllMonDF = GMonDF.copy()
        AllAnnDF = GAnnDF.copy()
    else:
        AllMonDF = AllMonDF.merge( GMonDF, how='inner', left_index=True, right_index=True)
        AllAnnDF = AllAnnDF.merge( GAnnDF, how='inner', left_index=True, right_index=True)
    # the resampling is done so now want go through and get our counts of contiguous
    #  wet days and contiguous dry days. Also track the start date for the contiguous 
    #  series and track the total depth for wet series and the daily depth within the
    #  wet series.
    cNumDays = len( PreDF )
    inWet = False
    inDry = False
    cWetCnt = 0
    cDryCnt = 0
    DryList = list()
    WetList = list()
    for dD in range( cNumDays ):
        cTSInd = PreDF.index[dD]
        cDT = dt.datetime( cTSInd.year, cTSInd.month, cTSInd.day )
        if dD == 0:
            cWStartDT = cDT
            cDStartDT = cDT
        cPDepth = float( PreDF.at[cTSInd,'Precip_mmpd'] )
        if cPDepth >= WD_THRESH:
            # this is the wet day case
            if inWet:
                cWetCnt += 1
                totPrecip += cPDepth
                dayPreL.append( cPDepth )
            else:
                inWet = True
                inDry = False
                cWStartDT = cDT
                cWetCnt = 1
                dayPreL = [ cPDepth ]
                totPrecip = cPDepth
                if dD > 0:
                    DryList.append( [ cDStartDT, cDryCnt ] )
                    cDryCnt = 0
        else:
            # this is the dry day case
            if inDry:
                cDryCnt += 1
            else:
                inWet = False
                inDry = True
                cDStartDT = cDT
                cDryCnt = 1
                if dD > 0:
                    WetList.append( [ cWStartDT, cWetCnt, totPrecip, dayPreL ] )
                    cWetCnt = 0
                    totPrecip = 0.0
                    dayPreL = list()
        # end of outer depth if
    # end of the day for
    # check for the last entry
    if inWet:
        WetList.append( [ cWStartDT, cWetCnt, totPrecip, dayPreL ] )
    else:
        DryList.append( [ cDStartDT, cDryCnt ] )
    # add our state analysis lists to our dictionaries
    DryDict[GridInd] = DryList
    WetDict[GridInd] = WetList
# end of for loop

Now are ready to output our various items. Do the pickle files first

In [30]:
MonPCKF = os.path.normpath( os.path.join( OUT_DIR, "AllMonth_1981-2010.pickle" ) )
AllMonDF.to_pickle( MonPCKF )
AnnPCKF = os.path.normpath( os.path.join( OUT_DIR, "AllYears_1981-2010.pickle" ) )
AllAnnDF.to_pickle( AnnPCKF )

Next use the feather library for R compatibility

In [31]:
FAllMonDF = AllMonDF.copy()
FAllAnnDF = AllAnnDF.copy()
FAllMonDF = FAllMonDF.reset_index()
FAllAnnDF = FAllAnnDF.reset_index()

In [32]:
MonFeatherF = os.path.normpath( os.path.join( OUT_DIR, "AllMonth_1981-2010.feather" ) )
FAllMonDF.to_feather( MonFeatherF )
AnnFeatherF = os.path.normpath( os.path.join( OUT_DIR, "AllYears_1981-2010.feather" ) )
FAllAnnDF.to_feather( AnnFeatherF )

Finally output to a spreadsheet

In [33]:
OutXLSX = os.path.normpath( os.path.join( OUT_DIR, "Precip_Agg_1981-2010.xlsx" ) )
with pd.ExcelWriter(OutXLSX) as writer:
    GridDF.to_excel( writer, sheet_name="Grid_Metadata", na_rep=str(np.nan),
                     index=True, index_label="Id" )
    AllMonDF.to_excel( writer, sheet_name="Monthly", na_rep=str(np.nan),
                       index=True, index_label="DateTime_UTC" )
    AllAnnDF.to_excel( writer, sheet_name="Annual", na_rep=str(np.nan),
                       index=True, index_label="DateTime_UTC" )
# end of with and write output

## Wet and Dry Days

While the monthly and annual aggregation was being completed, also collated the contiguous wet and dry day counts. These are in a dictionary. Need to process these out so that can work with them further

In [56]:
AllKeys = GridDF['Grid_Index'].to_list()

Determine the maximum number of wet days

In [57]:
CompKeys = sorted( WetDict.keys() )
CompKeys == AllKeys

True

In [58]:
MaxWetDays = 0
TotWetSeqs = 0
for tKey in AllKeys:
    TotWetSeqs = TotWetSeqs + len( WetDict[tKey] )
    NewWetDays = max( [x[1] for x in WetDict[tKey]] )
    if NewWetDays > MaxWetDays:
        MaxWetDays = NewWetDays
# end of for
MaxWetDays, TotWetSeqs

(19, 226190)

Go through our dictionaries and create DataFrames for each grid cell and then concatenate these all together.

In [70]:
DDFList = list()
WDFList = list()

In [71]:
for tKey in AllKeys:
    tDryList = DryDict[tKey]
    dNEnts = len( tDryList )
    DataDict = { "Grid_Id" : [ tKey for x in range(dNEnts) ],
                 "Year" : [x[0].year for x in tDryList],
                 "Month" : [x[0].month for x in tDryList],
                 "Day" : [x[0].day for x in tDryList],
                 "Dry_Count" : [x[1] for x in tDryList], }
    tDryDF = pd.DataFrame( data=DataDict )
    #tDryDF["Month"] = tDryDF.apply( lambda row: ExIntMonth( row["Start_Date"] ), axis=1 )
    DDFList.append( tDryDF )
    tWetList = WetDict[tKey]
    wNEnts = len( tWetList )
    WDaysArray = np.zeros( (wNEnts, MaxWetDays), dtype=np.float32 )
    # fill in the wet days array
    for iI in range(wNEnts):
        wdsList = tWetList[iI][3]
        cNDays = len( wdsList )
        for jJ in range( cNDays ):
            cdDep = wdsList[jJ]
            WDaysArray[iI, jJ] = cdDep
        # end of days for
    # end of rows for
    # now can create our DataFrame
    DataDict = { "Grid_Id" : [ tKey for x in range(wNEnts) ],
                 "Year" : [x[0].year for x in tWetList],
                 "Month" : [x[0].month for x in tWetList],
                 "Day" : [x[0].day for x in tWetList],
                 "Wet_Count" : [x[1] for x in tWetList], 
                 "Total_Depth" : [x[2] for x in tWetList], }
    for dD in range(1, (MaxWetDays + 1)):
        DayLabel = "Day_%d" % dD
        DataDict[DayLabel] = WDaysArray[:, (dD-1)]
    # end of day label for
    tWetDF = pd.DataFrame( data=DataDict )
    #tWetDF["Month"] = tWetDF.apply( lambda row: ExIntMonth( row["Start_Date"] ), axis=1 )
    WDFList.append( tWetDF )
# end of outer for

In [72]:
WetDayDF = pd.concat( WDFList, ignore_index=True )
DryDayDF = pd.concat( DDFList, ignore_index=True )

In [73]:
len( WetDayDF ), len( DryDayDF )

(226190, 226400)

In [74]:
display( HTML( WetDayDF.head().to_html() ) )

,Grid_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19
0,662328,1981,1,6,1,1.31,1.31,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,662328,1981,1,9,1,7.96,7.96,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,662328,1981,1,11,1,0.32,0.32,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,662328,1981,1,14,1,0.70,0.70,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,662328,1981,1,17,3,12.92,1.08,5.27,6.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
display( HTML( WetDayDF.tail().to_html() ) )

,Grid_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19
226185,680607,2010,9,20,5,11.610000,1.500000,1.19,0.95,7.38,0.59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226186,680607,2010,9,26,1,47.200001,47.200001,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226187,680607,2010,10,12,2,9.770000,5.270000,4.50,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226188,680607,2010,12,23,2,2.250000,0.400000,1.85,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226189,680607,2010,12,29,1,1.080000,1.080000,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
display( HTML( DryDayDF.head().to_html() ) )

,Grid_Id,Year,Month,Day,Dry_Count
0,662328,1981,1,1,5
1,662328,1981,1,7,2
2,662328,1981,1,10,1
3,662328,1981,1,12,2
4,662328,1981,1,15,2


In [78]:
DryDayDF['Dry_Count'].sum() + WetDayDF['Wet_Count'].sum()

2300970

Now output to pickle files in case need to reload

In [77]:
DryPCKF = os.path.normpath( os.path.join( OUT_DIR, "DryDays_1981-2010.pickle" ) )
DryDayDF.to_pickle( DryPCKF )
WetPCKF = os.path.normpath( os.path.join( OUT_DIR, "WetDays_1981-2010.pickle" ) )
WetDayDF.to_pickle( WetPCKF )

And output to feather format for integration with R.

In [79]:
DryFeatherF = os.path.normpath( os.path.join( OUT_DIR, "DryDays_1981-2010.feather" ) )
DryDayDF.to_feather( DryFeatherF )
DryFeatherF = os.path.normpath( os.path.join( OUT_DIR, "WetDays_1981-2010.feather" ) )
WetDayDF.to_feather( DryFeatherF )